In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tsai.all as ts
import torch
from torch import nn
from my_utilities import time_frequency_analysis, mma_Partition, time_frequency_analysis_output_1s
from sklearn.linear_model import RidgeClassifierCV

from sklearn.metrics import confusion_matrix
import time

In [1]:
(20000-2000)/6000

3.0

In [3]:
srate = 2000
new_srate = 200
sec = 1 # time step length
ele = pd.read_csv('data/ele_4_60_hz.csv', header = None, index_col = None).to_numpy()
# shape = (4, 15, 2, 20000, 14)
# 去除第一秒的数据,且重采样到200hz
ele = np.reshape(
    ele, (20000,15,4,2,14), order = 'F').transpose([2,1,3,0,4])[:,:,:,srate::srate//new_srate,:]

# 分成头部与耳部，标注label
ear_ele = ele[...,:8].astype(np.float32).reshape(-1,int(200 * sec),8)
head_ele = ele[...,8:].astype(np.float32).reshape(-1,int(200 * sec),6)
# labels = np.repeat([0,1,2,3], ear_ele.shape[0]//4)

In [4]:
ear_ele.shape

(1080, 200, 8)

In [5]:
# np.roll(np.arange(24).reshape(3,8), 1)
def roll_window(x, y, stride = 100):
    '''
    args:
        x(sample, channel, time_step): numpy ndarray, must contain the axis which want to be rolled
        axis: int, the specific axis
        stride: int, step length
    '''
    x_temp = x.reshape(x.shape[0], x.shape[1],-1,stride)  # 改AXIS必须改这里
    strided_x = np.asarray([np.roll(x_temp, shift = i, axis = 2).reshape(x.shape[0], x.shape[1], -1)\
                            for i in np.arange(x_temp.shape[2])])
    y = np.tile(y, x_temp.shape[2])
    return strided_x.reshape(-1, x.shape[1], x.shape[2]), y

In [5]:
# batches, channels, time step
ear_ele = ele[...,:8].astype(np.float32).reshape(-1,int(200 * sec),8).transpose([0,2,1])
head_ele = ele[...,8:].astype(np.float32).reshape(-1,int(200 * sec),6).transpose([0,2,1])

# z-score
head_ele = (head_ele - head_ele.mean(axis = 2)[...,np.newaxis])\
                /head_ele.std(axis = 2)[...,np.newaxis]

ear_ele = (ear_ele - ear_ele.mean(axis = 2)[...,np.newaxis])\
                /ear_ele.std(axis = 2)[...,np.newaxis]

y = np.repeat([0,1,2,3], int(ear_ele.shape[0]/4))

################################ TFA #########################
print('begin to TFA')
mask = np.unique(np.array(list(range(5,59,5))+
                          list(range(7,59,7))+
                          list(range(9,59,9))+
                          list(range(11,59,11))))-2
# time frequency analysis for ear-EEG
temp_ear_ele = []
for i in ear_ele:
    _,_,temp = time_frequency_analysis(
        i.T, srate = new_srate, max_freq = 60, min_freq = 2, num_freq = 59, fwhm = 1)
    # _,_,temp = time_frequency_analysis_output_1s(
    #     i.T, srate = new_srate, max_freq = 60, min_freq = 2, num_freq = 59, fwhm = 1)    
    temp_ear_ele.append(temp)
    
# if use mask
ear_ele = np.asarray(temp_ear_ele)[:,mask,:]

# # if do not use mask
# ear_ele = np.asarray(temp_ear_ele)
del temp_ear_ele
##############################################################


ear_X_train, ear_X_test, ear_y_train, ear_y_test = train_test_split(ear_ele, y, test_size=0.1)
# head_X_train, head_X_test, head_y_train, head_y_test = train_test_split(head_ele, y, test_size=0.1)
# ################# DATA generate ##############################
# ear_X_train, ear_y_train = roll_window(ear_X_train, ear_y_train)
# ear_X_train, ear_y_train = shuffle(ear_X_train, ear_y_train)
##############################################################
ear_ele, ear_y, ear_splits = ts.combine_split_data([ear_X_train, ear_X_test], [ear_y_train, ear_y_test])

begin to TFA


In [6]:
ear_X_train.shape

(972, 27, 200)

In [ ]:
# batch_tfms = [ts.TSStandardize(by_sample=True)]
# tfms  = [None, [ts.Categorize()]]
# dls = ts.get_ts_dls(ear_ele, ear_y, 
#                  splits=ear_splits, 
#                  tfms=tfms, drop_last=False, 
#                  shuffle_train=False, batch_tfms=batch_tfms, bs=10_000)

# model = ts.build_ts_model(ts.ROCKET, dls=dls)

# X_train, y_train = ts.create_rocket_features(dls.train, model)
# X_valid, y_valid = ts.create_rocket_features(dls.valid, model)
# X_train.shape, X_valid.shape

In [4]:
def cross_validation(x, y, cv_number = 20, test_size = 0.1):
    acc = []
    batch_tfms = [ts.TSStandardize(by_sample=True)]
    tfms  = [None, [ts.Categorize()]]
    cfm = np.zeros([4,4])
    for _ in np.arange(cv_number):
        x, y = shuffle(x, y)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
            
        ear_X, ear_y, splits = ts.combine_split_data([X_train, X_test], [y_train, y_test])
        dls = ts.get_ts_dls(ear_X, ear_y, 
                         splits=splits, 
                         tfms=tfms, drop_last=False, 
                         shuffle_train=False, batch_tfms=batch_tfms, bs=10_000)        
        model = ts.build_ts_model(ts.ROCKET, dls=dls)
        
        X_train, y_train = ts.create_rocket_features(dls.train, model)
        X_valid, y_valid = ts.create_rocket_features(dls.valid, model)
        X_train.shape, X_valid.shape
    
        ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
        ridge.fit(X_train, y_train)
        print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')
        acc.append(ridge.score(X_valid, y_valid))
        
        cfm += confusion_matrix(y_valid, ridge.predict(X_valid))
        print(cfm)
    return acc, cfm

In [13]:
# np.asarray(acc_total).mean(axis = 1)

array([0.7       , 0.76041667, 0.68958333, 0.68958333, 0.70833333,
       0.71666667, 0.68333333, 0.69166667, 0.67916667, 0.67916667])

In [12]:
np.asarray(acc_total).mean(axis = 1)
np.asarray(acc_total_2).mean(axis = 1)

array([0.4625    , 0.45694444, 0.45833333, 0.45648148, 0.45601852,
       0.48981481, 0.45046296, 0.45601852, 0.4462963 , 0.4837963 ])

In [11]:

# pd.DataFrame({'number': np.arange(1,21)}).to_csv('result\ROCKET_ACC_01_13.csv', index = 0)


In [ ]:
srate = 2000
new_srate = 200
ele_original = pd.read_csv('data/ele_4_60_hz.csv', header = None, index_col = None).to_numpy()
# shape = (4, 15, 2, 20000, 14)

mask = np.unique(np.array(list(range(5,59,5))+
                          list(range(7,59,7))+
                          list(range(9,59,9))+
                          list(range(11,59,11))))-2
data = []
total_acc = []
for time_in_sec in [4]:
    sec = time_in_sec # time step length
    if 9%time_in_sec ==0:
        ele = np.reshape(
            ele_original, (20000,15,4,2,14), order = 'F').\
                transpose([2,1,3,0,4])[:,:,:,srate::srate//new_srate,:]
    else:
        ele = np.reshape(
            ele_original, (20000,15,4,2,14), order = 'F').\
                transpose([2,1,3,0,4])[:,:,:,srate:-srate:srate//new_srate,:]
        
    ear_ele = ele[...,:8].astype(np.float32).reshape(-1,int(200 * sec),8).transpose([0,2,1])
    y = np.repeat([0,1,2,3], int(ear_ele.shape[0]/4))
    
    # z-score
    ear_ele = (ear_ele - ear_ele.mean(axis = 2)[...,np.newaxis])\
                    /ear_ele.std(axis = 2)[...,np.newaxis]




    # time frequency analysis for ear-EEG
    temp_ear_ele = []
    for flame in ear_ele:
        _,_,temp = time_frequency_analysis(
            flame.T, srate = new_srate, max_freq = 60, min_freq = 2, num_freq = 59, fwhm = 0.75 * time_in_sec)
        # _,_,temp = time_frequency_analysis_output_1s(
        #     i.T, srate = new_srate, max_freq = 60, min_freq = 2, num_freq = 59, fwhm = 1)    
        temp_ear_ele.append(temp)
    
    # if use mask
    ear_ele = np.asarray(temp_ear_ele)
    # print(ear_ele.shape)
    temp_acc, cf_matrix = cross_validation(ear_ele, y, cv_number = 20, test_size = 0.1)
#     pd.concat([pd.read_csv(r'result/ROCKET_ACC_01_13.csv', header = 0),pd.DataFrame(temp_acc, columns=[f'{time_in_sec}s'])], 
#           axis = 1).to_csv(r'result/ROCKET_ACC_01_13.csv', index= None)
    
#     pd.DataFrame(cf_matrix).to_csv(
#         r'C:\MY\matlab\SSVEP\seesion2\result\ROCKET_cfmatrix_01_13.csv', 
#         mode = 'a', index = None, header = None)
    total_acc.append(temp_acc)

alpha: 1.00E+01  train: 0.99537  valid: 0.70833
[[2. 1. 1. 0.]
 [0. 7. 1. 0.]
 [0. 3. 2. 1.]
 [0. 0. 0. 6.]]
alpha: 1.00E+01  train: 0.99537  valid: 0.62500
[[ 3.  2.  1.  1.]
 [ 1. 13.  1.  1.]
 [ 1.  3.  5.  3.]
 [ 2.  0.  0. 11.]]
alpha: 1.00E+01  train: 0.99537  valid: 0.75000
[[ 3.  2.  2.  1.]
 [ 1. 20.  2.  1.]
 [ 2.  4.  6.  4.]
 [ 3.  0.  0. 21.]]
alpha: 1.00E+01  train: 0.99537  valid: 0.79167
[[ 7.  3.  2.  2.]
 [ 2. 24.  3.  1.]
 [ 2.  4. 12.  5.]
 [ 3.  0.  0. 26.]]
alpha: 1.00E+01  train: 0.99537  valid: 0.75000
[[13.  4.  3.  2.]
 [ 2. 30.  3.  2.]
 [ 2.  4. 16.  5.]
 [ 5.  0.  1. 28.]]
alpha: 1.00E+01  train: 0.99537  valid: 0.79167
[[18.  4.  3.  2.]
 [ 3. 36.  3.  2.]
 [ 3.  5. 18.  6.]
 [ 5.  1.  1. 34.]]


KeyboardInterrupt: 